In [ ]:
# Initialize Otter
import otter
grader = otter.Notebook("lab.ipynb")

# Lab 5 – Imputation

## DSC 80, Spring 2022

### Due Date: Monday, May 2nd at 11:59 PM

## Instructions
Much like in DSC 10, this Jupyter Notebook contains the statements of the problems and provides code and Markdown cells to display your answers to the problems. Unlike DSC 10, the notebook is *only* for displaying a readable version of your final answers. The coding will be done in an accompanying `lab.py` file that is imported into the current notebook.

Labs and programming assignments will be graded in (at most) two ways:
1. The functions and classes in the accompanying `lab.py` file will be tested (a la DSC 20),
2. The notebook may be graded (if it contains free response questions or asks you to draw plots).

**Do not change the function names in the `lab.py` file!**
- The functions in the `lab.py` file are how your assignment is graded, and they are graded by their name.
- If you changed something you weren't supposed to, just use git to revert! Ask us if you need help with this, or google around for `git revert`.

**Tips for working in the notebook**:
- The notebooks serve to present the questions and give you a place to present your results for later review.
- The notebooks in *lab assignments* are not graded (only the `lab.py` file is submitted and graded).
- The notebook serves as a nice environment for 'pre-development' and experimentation before designing your function in your `lab.py` file. You can write code here, but make sure that all of your real work is in the `lab.py` file.

**Tips for developing in the `lab.py` file**:
- Do not change the function names in the starter code; grading is done using these function names.
- Do not change the docstrings in the functions. These are there to tell you if your work is on the right track!
- You are encouraged to write your own additional helper functions to solve the lab! 
- Always document your code!

### Importing code from `lab.py`

* We import our `lab.py` file that's contained in the same directory as this notebook.
* We use the `autoreload` notebook extension to make changes to our `lab.py` file immediately available in our notebook. Without this extension, we would need to restart the notebook kernel to see any changes to `lab.py` in the notebook.
    - `autoreload` is necessary because, upon import, `lab.py` is compiled to bytecode (in the directory `__pycache__`). Subsequent imports of `lab` merely import the existing compiled python.

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from lab import *

In [ ]:
import os
import pandas as pd
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt

import requests
import bs4

***Note:*** While working on the lab, check the Campuswire post titled "Lab 5 Released!" for any clarifications.

## Question 1 – Payment Data 💰

In `data/payment.csv`, you are given a dataset of payment information for purchases made on January 1st, 2019. The dataset contains the purchaser's `'Id'`, `'credit_card_type'`, `'credit_card_number'`, and `'date_of_birth'`.

Your job is to assess the missingness in the payments data. The question you need to answer is, **is `'credit_card_number'` missing at random dependent on the age of the shopper?**. Let's approach this question in two ways.

#### `first_round`

Look at distribution of ages by missingness of `'credit_card_number'` and determine if the missingness is dependent on age or not.

Use the following steps to approach this problem:

- Compute the ages of the purchasers.
- Draw the distribution of ages by missingness of `'credit_card_number'`. Specifically, you will draw two histograms or density curves:
    - One of ages where `'credit_card_number'` is missing.
    - One of ages where `'credit_card_number'` is not missing.
- Perform a permutation test for whether or not the two distributions mentioned above are drawn from the same population distribution. Use a 5% significance level. Use the **absolute difference of means** as your test statistic.

Write a function `first_round` with no arguments that returns a **list** with two values:
* The first value is the p-value from your permutation test. 
* The second value is either `'R'` if you reject the null hypothesis, or `'NR'` if you fail to reject the null.

**Does the result match your guess? If not, what might be a problem?**

***Hint:*** 
- Look at [Lecture 12](https://dsc80.com/resources/lectures/lec12/lec12.html#Assessing-missingness-through-data).
- To find a customer's age, compute the number of years between their birth year and 2022.

<br>

#### `second_round`

Run another permutation test for the empirical distribution of age conditional on `'credit_card_number'` with a 5% significance level. This time, use the **Kolmogorov-Smirnov statistic** as your test statistic.

Write a function `second_round` with no arguments that returns a __list__ with three values: 
* The first value is the p-value from your new permutation test.
* The second value is either `'R'` if you reject the null hypothesis, or `'NR'` if you fail to reject the null. 
* The third value is your final conclusion: `'D'` (the missingness of `'credit_card_number'` is dependent on age) or `'ND'` (the missingness of `'credit_card_number'` is not dependent on age).

***Hint:*** 
- In both [Lecture 12](https://dsc80.com/resources/lectures/lec12/lec12.html#Assessing-missingness-through-data) and [Lecture 13](https://dsc80.com/resources/lectures/lec13/lec13.html), we ran permutation tests using the Kolmogorov-Smirnov test statistic **without `for`-loops**. You can use this same procedure; we have already imported `stats` from `scipy`.

In [ ]:
# Run your permutation tests in the Jupyter Notebook;
# put your final results in lab.py
payments_fp = os.path.join('data', 'payment.csv')
payments = pd.read_csv(payments_fp)
payments.head()

In [ ]:
# don't change this cell, but do run it -- it is needed for the tests to work
first_pval, first_result = first_round()
second_pval, second_result, second_result1 = second_round()

In [ ]:
grader.check("q1")

## Question 2 – Missing Heights 🕵️

In the file `data/missing_heights.csv` are the heights of children and their fathers (`'child'` and `'father'`). The `'child_X'` columns are missing values in varying proportions; for each X, `'child_X'` is X\% not missing (and hence (100-X)\% missing). The missingness of these `'child_X'` columns were created as MAR dependent on `'father'` height (similar to what was done in Lecture 12). The missingness of these `'child_X'` columns are all equally dependent on father height.

You will attempt to **verify** the missingness of the `'child_X'` columns as being dependent on the `'father'` height column by using permutation tests. Your permutation tests should use the Kolmogorov-Smirnov test statistic. You can use `scipy.stats`' built-in K-S function to run your permutation tests and compute your p-values; you don't need to simulate manually using a `for`-loop.

#### `verify_child`

Write a function `verify_child` that takes in the `heights` DataFrame and returns a Series of p-values from your permutation tests, indexed by the columns `'child_X'`.

***Notes:*** 

- Unlike in Question 1, your permutation tests should run within your `verify_child` function.
- You may loop over the **columns** of `heights`.

<br>

#### `missing_data_amounts`

Now, interpret your results. In the function `missing_data_amounts`, return a __list__ of correct statements from the options below:
1. The p-value for `'child_50'` is small because the *sampling distribution* of test statistics has low variance.
2. MAR is hardest to determine when there are very different proportions of missing and not-missing values.
3. The difference between p-values for `'child_5'` and `'child_95'` is due to randomness.
4. We should expect the p-value of `'child_X'` and `'child_(100-X)'` to be similar.
5. We should only expect the p-value of `'child_X'` and `'child_(100-X)'` to be similar if the columns are MCAR.

In [ ]:
# don't change this cell, but do run it -- it is needed for the tests to work
heights_fp = os.path.join('data', 'missing_heights.csv')
heights = pd.read_csv(heights_fp)
out1_q2 = verify_child(heights.copy())
out2_q2 = missing_data_amounts()

In [ ]:
grader.check("q2")

## Question 3 – Imputing Heights 🧍📏

In [Lecture 13](https://dsc80.com/resources/lectures/lec13/lec13.html), you learned how to perform single-valued imputation conditionally on a **categorical** column: impute with the mean for each group. That is, for each distinct value of the **categorical** column, there is a single imputed value.

Here, you will perform single-valued imputation conditionally on a **quantitative** column. 

You will work with a version of the `heights` DataFrame, `new_heights`, that has a `'father'` column and a single `'child'` column. The `'child'` column has missing values. To impute the `'child'` column, transform the `'father'` column into a categorical column by binning the values of `'father'` into [quartiles](https://en.wikipedia.org/wiki/Quartile). Once this is done, you can impute `'child'` as in lecture (and described above).

<br>

#### `cond_single_imputation`

Write a function `cond_single_imputation` that takes in a DataFrame with columns `'father'` and `'child'` (where `'child'` has missing values) and performs a single-valued mean imputation of the `'child'` column, conditional on `'father'`. Your function should return a **Series**.

***Hints:*** 
- `pd.qcut` may be helpful!
- The `transform` is useful for this question (see [Lecture 7](https://dsc80.com/resources/lectures/lec07/lec07.html#The-transform-method)), though it's also possible using `aggregate`. 
- As a reminder, *loops are not allowed*, and functions mentioned in "Hints" are not required.

In [ ]:
# don't change this cell, but do run it -- it is needed for the tests to work
heights_fp = os.path.join('data', 'missing_heights.csv')
new_heights = pd.read_csv(heights_fp)[['father', 'child_50']]
new_heights = new_heights.rename(columns={'child_50': 'child'})
out_q3 = cond_single_imputation(new_heights.copy())

In [ ]:
grader.check("q3")

## Question 4 – Probabilistic Imputation 🎲

In [Lecture 13](https://dsc80.com/resources/lectures/lec13/lec13.html#Probabilistic-imputation), you learned how to impute a quantitative column by sampling from the observed values. **One problem with this technique is that the imputation will never generate imputed values that weren't already in the dataset.** For example, 57, 57.5, and 59 are values in the `'child'` column of `new_heights` while 58 is not. Thus, any imputation done by sampling from the observed values in the `'child'` column will not be able to generate a height of 58, even though it's clearly a reasonable value to occur in the dataset.

To keep things simple, you will impute the `'child'` column **unconditionally** from the distribution of `'child'` heights present in the dataset. This means that you will use the values present in `'child'` to impute missing values, without looking at other columns.

An approach to quantitative imputation that overcomes the limitation mentioned above is as follows:
- Create a histogram of observed `'child'` heights, using 10 bins.
- Use the histogram to generate a number within the observed range of `'child'` heights:
    - The likelihood a generated number belongs to a given bin is equal to the area of that bin. (Remember, in histograms, areas are proportions.)
    - Any number within a fixed bin is equally likely to occur.
    
Let's illustrate this approach with an example. Let `demo` be the array of 10 numbers defined below.

```py
demo = np.array([10, 11, 11, 13, 14, 14, 13.5, 14, 15, 16])
```

- The first step is creating a histogram of `demo`. Note that with this small dataset, we will use 3 bins, but you will be using 10 bins in your imputation process.

<img src='imgs/demo_histogram.png' width=300>

- Note that in your process, you don't actually need to draw a histogram – instead, use `np.histogram`.
- In the histogram above, we see that $2 \cdot 0.15 = 0.3 = 30\%$ of values lie in the [10, 12) bin, $2 \cdot 0.1 = 0.2 = 20\%$ of values lie in the [12, 14) bin, and $2 \cdot 0.25 = 0.5 = 50\%$ of values lie in the [14, 16] bin.
- Next, we need to pick a bin at random. There's a 30\% chance we pick the [10, 12) bin, a 20\% chance we pick the [12, 14) bin, and a 50\% chance we pick the [14, 16] bin. `np.random.choice` will be helpful in picking a bin at random.
- Once we pick a bin, we pick a number **uniformly at random** from within the bin. For instance, suppose we randomly chose the [14, 16] bin in the previous step. We then must select a (real) number between 14 and 16 uniformly at random. `np.random.uniform` can help you here.

<br>

#### `quantitative_distribution`
    
Create a function `quantitative_distribution` that takes in a Series, `child`, in which some values are missing, and an integer `N > 0`, and returns an **array** of `N` imputed values using the method described above. 

***Note:*** You may use a `for`-loop.

<br>

#### `impute_height_quant`

Create a function `impute_height_quant` that takes in a Series, `child`, in which some values are missing and imputes them using the scheme above. `impute_height_quant` should return a Series that is the same length of `child` but with no missing values. **You should use `quantitative_distribution` to help you do this.**

In [ ]:
# don't change this cell, but do run it -- it is needed for the tests to work
heights_fp = os.path.join('data', 'missing_heights.csv')
heights = pd.read_csv(heights_fp)
child = heights['child_50']
quantitative_distribution_out_q4 = quantitative_distribution(child.copy(), 100)
impute_height_quant_out_q4 = impute_height_quant(child.copy())

In [ ]:
grader.check("q4")

## Question 5 – The Rules of Web Scraping 🚔

In Lecture 14, we will start studying web scraping. This question will give you an introduction to the rules behind it.

Many sites have a published policy allowing or disallowing automatic access to their site. Often, this policy is in a text file named `robots.txt`. [Here is a good article](https://moz.com/learn/seo/robotstxt) that explains what these files are, where to find them, and how to use them. **After reading the article**, please answer the following questions.

<br>

### Multiple-Choice Questions

**1. What is the purpose of `robots.txt`?**

1. To informs agents which pages to crawl.

2. To informs agents that the site is automated.

3. To inform agents that robots will chase them down if their info is stolen.


**2. Where do you put your `robots.txt` file?**

1. In the folder you want to disallow.

2. In the root directory of your website.

3. In a Google search.

**3. Is it illegal to scrape a site if there is no `robots.txt` present?** (***Hint:*** [Read this](https://medium.com/@tjwaterman99/web-scraping-is-now-legal-6bf0e5730a78) if you're not sure. If you aren't able to access the article because Medium says "You've read all your free member-only stories...", try opening the article in a new incognito or private browsing window 😅.)

1. Yes.

2. No.

**4. Can each subdomain on a root domain use separate `robots.txt` files?**

1. Yes.

2. No.


<br>


### Website Hunt

Next, **find three websites that explicitly use a `robots.txt` file and allow scraping by everyone, and three that do not allow scraping by generic user-agents**.

When browsing through `robots.txt` files, you may notice that some have entries for several different user-agents. The user-agent you are interested in for this question is the generic user-agent, which is denoted by `*`. A `/` after a `Disallow:` indicates that no scraping is allowed, whereas nothing after the `Disallow:` indicates that all scraping is allowed.

***Notes:***
- During your search, you may notice that very few websites allow scraping by everyone. When trying to find websites that satisfy this criteria, you are encouraged to think contextually about what kinds of websites would and wouldn't mind you scraping their data. Would a government website likely mind scraping by everyone? How about a website someone created just for fun?

- Some websites may cause Gradescope to time out. Please change a website if you encounter this issue. 

- Below, you are asked to store the URLs of the websites you find in a list. When storing URLs, you can add `'/robots.txt'` to the end, but you don't have to – either format will be accepted.

<br>


#### `answers`

Create an argument-free function `answers` that returns **two lists**:

* one containing your answers to the multiple-choice questions, and
* one containing the URLs of the sites you found. The first 3 URLs in the list should be of websites that allow scraping by everyone, and the last 3 URLs in the list should be of websits that disallow scraping by generic user-agents.

In [ ]:
# don't change this cell, but do run it -- it is needed for the tests to work
mc_answers, websites = answers()

In [ ]:
grader.check("q5")

## Congratulations! You're done! 🏁

Submit your `lab.py` file to Gradescope. Note that you only need to submit the `lab.py` file; this notebook should not be uploaded.

Before submitting, you should ensure that all of your work is in the `lab.py` file. You can do this by running the doctests below, which will verify that your work passes the public tests **and** that your work is in the `lab.py` file. Run the cell below; you should see no output.

In [ ]:
!python -m doctest lab.py

In addition, `grader.check_all()` will verify that your work passes the public tests.

---

To double-check your work, the cell below will rerun all of the autograder tests.

In [ ]:
grader.check_all()